### Module import

In [1]:
import os
import cv2
import time
import datetime
import random
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from torchvision import models
from sklearn.metrics import f1_score
from torch.utils.data import Dataset, DataLoader

!pip install timm
import timm

!pip install -q -U albumentations
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

!pip install madgrad
from madgrad import MADGRAD

In [2]:
timm.list_models()

['adv_inception_v3',
 'cspdarknet53',
 'cspdarknet53_iabn',
 'cspresnet50',
 'cspresnet50d',
 'cspresnet50w',
 'cspresnext50',
 'cspresnext50_iabn',
 'darknet53',
 'densenet121',
 'densenet121d',
 'densenet161',
 'densenet169',
 'densenet201',
 'densenet264',
 'densenet264d_iabn',
 'densenetblur121d',
 'dla34',
 'dla46_c',
 'dla46x_c',
 'dla60',
 'dla60_res2net',
 'dla60_res2next',
 'dla60x',
 'dla60x_c',
 'dla102',
 'dla102x',
 'dla102x2',
 'dla169',
 'dm_nfnet_f0',
 'dm_nfnet_f1',
 'dm_nfnet_f2',
 'dm_nfnet_f3',
 'dm_nfnet_f4',
 'dm_nfnet_f5',
 'dm_nfnet_f6',
 'dpn68',
 'dpn68b',
 'dpn92',
 'dpn98',
 'dpn107',
 'dpn131',
 'eca_vovnet39b',
 'ecaresnet26t',
 'ecaresnet50d',
 'ecaresnet50d_pruned',
 'ecaresnet50t',
 'ecaresnet101d',
 'ecaresnet101d_pruned',
 'ecaresnet200d',
 'ecaresnet269d',
 'ecaresnetlight',
 'ecaresnext26t_32x4d',
 'ecaresnext50t_32x4d',
 'efficientnet_b0',
 'efficientnet_b1',
 'efficientnet_b1_pruned',
 'efficientnet_b2',
 'efficientnet_b2_pruned',
 'efficientnet_b

### CFG

In [3]:
CFG = {
    "model" : "tf_efficientnet_b4",
    "pretrained" : True,
    "drop_rate" : 0.2,
    "seed" : 9,
    "img_size" : (380, 380),
    "crop_size" : (380, 380),
    "epoch" : 4,
    "T_0" : 10,
    "lr" : 1e-4,
    "min_lr" : 1e-6,
    "weight_decay" : 1e-2,
    "train_bs" : 32,
    "test_bs" : 32,
    "cross_valid" : True,
    "k_fold" : 5,
    "num_workers" : 8,
    "num_accum" : 1,
    "num_classes" : 18,
    "device" : torch.device("cuda")
}

### Transforms

In [4]:
transforms = {
    "train" : A.Compose([
#         A.Resize(CFG["img_size"][0], CFG["img_size"][1], p=1.0),
        A.CenterCrop(CFG["crop_size"][0], CFG["crop_size"][1], p=1.0),
        A.CLAHE(p=1.0),
        A.HorizontalFlip(p=0.5),
        A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),   # 색체 변경
        A.RandomBrightnessContrast(p=0.2),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], p=1.0),
        ToTensorV2(p=1.0)
        ]),
    "test" : A.Compose([
#         A.Resize(CFG["img_size"][0], CFG["img_size"][1], p=1.0),
        A.CenterCrop(CFG["crop_size"][0], CFG["crop_size"][1], p=1.0),
        A.CLAHE(p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], p=1.0),
        ToTensorV2(p=1.0)
        ])
}

### Loss Function

In [5]:
class FocalLoss(nn.Module):

    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha, (float, int)): self.alpha = torch.Tensor([alpha, 1 - alpha])
        if isinstance(alpha, list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0), input.size(1), -1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1, 2)                         # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1, input.size(2))    # N,H*W,C => N*H*W,C
        target = target.view(-1, 1)

        logpt = F.log_softmax(input, dim=1)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = logpt.exp()

        if self.alpha is not None:
            if self.alpha.type() != input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0, target.data.view(-1))
            logpt = logpt * at

        loss = -1 * (1 - pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

In [6]:
# -- Label Smoothing Loss
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes=3, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [7]:
# -- F1 Loss
# https://gist.github.com/SuperShinyEyes/dcc68a08ff8b615442e3bc6a9b55a354
class F1Loss(nn.Module):
    def __init__(self, classes=3, epsilon=1e-7):
        super().__init__()
        self.classes = classes
        self.epsilon = epsilon
    def forward(self, y_pred, y_true):
        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        y_true = F.one_hot(y_true, self.classes).to(torch.float32)
        y_pred = F.softmax(y_pred, dim=1)

        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2 * (precision * recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1 - self.epsilon)
        return 1 - f1.mean()

### Model Setting

In [8]:
model = timm.create_model(CFG["model"], pretrained=CFG["pretrained"], num_classes=CFG["num_classes"], drop_rate=CFG["drop_rate"])
model.to(CFG["device"])
optimizer = torch.optim.Adam(model.parameters(), lr=CFG["lr"], weight_decay=CFG["weight_decay"])
# optimizer = MADGRAD(model.parameters(), lr=CFG["lr"], weight_decay=CFG["weight_decay"])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=CFG["T_0"], eta_min=CFG["min_lr"])
criterion = FocalLoss().to(CFG["device"])

### Dataset

In [9]:
train_dir = "/opt/ml/input/data/train"
test_dir = "/opt/ml/input/data/eval"

train_csv = os.path.join(train_dir, "fixed_label.csv")
test_csv = os.path.join(test_dir, "info.csv")

df = pd.read_csv(train_csv)

In [10]:
class MaskDataset(Dataset):
    def __init__(self, df, exist_label, transforms=None):
        self.df = df
        self.transforms = transforms
        self.exist_label = exist_label
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index: int):
        if self.exist_label:
            target = self.df.iloc[index]["label"]       
            path = self.df.iloc[index]["filepath"]
        else:
            path = os.path.join(test_dir, "images", self.df.iloc[index]["ImageID"])
            
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
        
        if self.exist_label:
            return img, target
        else:
            return img

### Metric Function

In [11]:
def eval_accuracy(model, iterator):
    with torch.no_grad():
        model.eval()
       
        n_total, n_correct = 0, 0
        for batch_data, batch_label in tqdm(iterator):
            target = batch_label.to(CFG["device"])
            output = model(batch_data.float().to(CFG["device"]))
            _, prediction = torch.max(output.data, 1)
            n_correct += (prediction == target).sum().item()
            n_total += batch_data.size(0)
        accuracy = n_correct / n_total
        
    return accuracy

In [12]:
def eval_f1_score(model, iterator):
    with torch.no_grad():
        model.eval()
        
        targets, predictions = [], []
        for batch_data, batch_label in tqdm(iterator):
            target = batch_label.to(CFG["device"])
            output = model(batch_data.float().to(CFG["device"]))
            _, prediction = torch.max(output.data, 1)
            targets += target.tolist()
            predictions += prediction.tolist()
            
    return f1_score(targets, predictions, average="macro")

### Train

In [13]:
def train_one_epoch(CFG, model, criterion, train_iter, optimizer, scheduler):
    model.train()
    optimizer.zero_grad()
    
    for j, (batch_data, batch_label) in tqdm(enumerate(train_iter), total=len(train_iter), position=0, leave=True):
        target = batch_label.to(CFG["device"])
        prediction = model(batch_data.float().to(CFG["device"]))
        loss = criterion(prediction, target)

        loss.backward()

        # gradient accumulation
        if j % CFG["num_accum"] == 0:
            optimizer.step()
            optimizer.zero_grad()
        
    if scheduler is not None:
        scheduler.step()

In [14]:
def valid_one_epoch(CFG, model, criterion, valid_iter):
    with torch.no_grad():
        model.eval()
        
        for batch_data, batch_label in tqdm(valid_iter):            
            target = batch_label.to(CFG["device"])
            prediction = model(batch_data.float().to(CFG["device"]))
            loss = criterion(prediction, target)

In [15]:
def train(CFG, transforms, df, save_path):
    groups = [df for _, df in df.groupby("id")]
    random.seed(CFG["seed"])
    random.shuffle(groups)
    df = pd.concat(groups)
        
    best_score = 0
    
    train_df = df.iloc[:len(df)*4//5]
    valid_df = df.iloc[len(df)*4//5:]
    
    train_dataset = MaskDataset(train_df, exist_label=True, transforms=transforms["train"])
    valid_dataset = MaskDataset(valid_df, exist_label=True, transforms=transforms["test"])

    train_iter = DataLoader(dataset=train_dataset, batch_size=CFG["train_bs"], shuffle=True, num_workers=CFG["num_workers"])
    valid_iter = DataLoader(dataset=valid_dataset, batch_size=CFG["test_bs"], shuffle=False, num_workers=CFG["num_workers"])

    for i in range(CFG["epoch"]):
        train_one_epoch(CFG, model, criterion, train_iter, optimizer, scheduler)
        valid_one_epoch(CFG, model, criterion, valid_iter)

        train_score = eval_f1_score(model, train_iter)
        valid_score = eval_f1_score(model, valid_iter)

        # save
        if valid_score > best_score:            
            best_score = valid_score
            torch.save(model.state_dict(), os.path.join(save_path, f'{CFG["model"]}.pth'))
            print("(BEST)", end=" ")

        print(f'epoch:[{i + 1}/{CFG["epoch"]}] train_score:[{train_score * 100 :.4f}%] valid_score:[{valid_score * 100 :.4f}%]')

In [16]:
def cross_valid(CFG, transforms, df, save_path):
    groups = [df for _, df in df.groupby("id")]
    random.seed(CFG["seed"])
    random.shuffle(groups)
    df = pd.concat(groups)
    
    total_size = len(df)
    fraction = 1 / CFG["k_fold"]
    seg = int(total_size * fraction)
    # tr:train,val:valid; r:right,l:left;  eg: trrr: right index of right side train subset 
    # index: [trll,trlr],[vall,valr],[trrl,trrr]
    for k in range(CFG["k_fold"]):
        model = timm.create_model(CFG["model"], pretrained=CFG["pretrained"], num_classes=CFG["num_classes"], drop_rate=CFG["drop_rate"])
        model.to(CFG["device"])
        optimizer = torch.optim.Adam(model.parameters(), lr=CFG["lr"], weight_decay=CFG["weight_decay"])
#         optimizer = MADGRAD(model.parameters(), lr=CFG["lr"], weight_decay=CFG["weight_decay"])
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=CFG["T_0"], eta_min=CFG["min_lr"])
        criterion = FocalLoss().to(CFG["device"])
        
        best_score = 0
        
        trll = 0
        trlr = k * seg
        vall = trlr
        valr = k * seg + seg
        trrl = valr
        trrr = total_size

        train_left_df = df.iloc[trll:trlr]
        train_right_df = df.iloc[trrl:trrr]
        train_df = pd.concat([train_left_df, train_right_df])
        valid_df = df.iloc[vall:valr]
        
        train_dataset = MaskDataset(train_df, exist_label=True, transforms=transforms["train"])
        valid_dataset = MaskDataset(valid_df, exist_label=True, transforms=transforms["test"])

        train_iter = DataLoader(dataset=train_dataset, batch_size=CFG["train_bs"], shuffle=True, num_workers=CFG["num_workers"])
        valid_iter = DataLoader(dataset=valid_dataset, batch_size=CFG["test_bs"], shuffle=False, num_workers=CFG["num_workers"])
        
        for i in range(CFG["epoch"]):
            train_one_epoch(CFG, model, criterion, train_iter, optimizer, scheduler)
            valid_one_epoch(CFG, model, criterion, valid_iter)
            
            train_score = eval_f1_score(model, train_iter)
            valid_score = eval_f1_score(model, valid_iter)
            
            # save
            if valid_score > best_score:            
                best_score = valid_score
                torch.save(model.state_dict(), os.path.join(save_path, f'{CFG["model"]}_[{k}].pth'))
                print("(BEST)", end=" ")
                
            print(f'k_fold:[{k + 1}/{CFG["k_fold"]}] epoch:[{i + 1}/{CFG["epoch"]}] train_score:[{train_score * 100 :.4f}%] valid_score:[{valid_score * 100 :.4f}%]')

In [17]:
start = time.time()
save_path = "/opt/ml/code/checkpoints"
if CFG["cross_valid"]:
    cross_valid(CFG, transforms, df, save_path)
else:
    train(CFG, transforms, df, save_path)
print(str(datetime.timedelta(seconds=time.time()-start)).split(".")[0])

100%|██████████| 118/118 [00:18<00:00,  6.55it/s]


(BEST) k_fold:[1/5] epoch:[1/4] train_score:[90.6329%] valid_score:[75.8169%]


100%|██████████| 118/118 [00:18<00:00,  6.34it/s]

k_fold:[1/5] epoch:[2/4] train_score:[95.2375%] valid_score:[75.5610%]



100%|██████████| 118/118 [00:18<00:00,  6.40it/s]


...

(BEST) k_fold:[5/5] epoch:[2/4] train_score:[95.9663%] valid_score:[76.6276%]


100%|██████████| 118/118 [00:18<00:00,  6.27it/s]

k_fold:[5/5] epoch:[3/4] train_score:[93.2236%] valid_score:[74.7978%]



100%|██████████| 118/118 [00:18<00:00,  6.42it/s]


(BEST) k_fold:[5/5] epoch:[4/4] train_score:[96.5073%] valid_score:[78.6507%]
1:50:50


### Inference

In [ ]:
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))

test_dataset = MaskDataset(submission, exist_label=False, transforms=transforms["test"])
test_iter = DataLoader(test_dataset, batch_size=CFG["test_bs"], shuffle=False, num_workers=CFG["num_workers"])

all_predictions = []

if CFG["cross_valid"]:
    for k in range(CFG["k_fold"]):
        model.load_state_dict(torch.load(os.path.join(save_path, f'{CFG["model"]}_[{k}].pth')))
        model.eval()
        temp_predictions = []
        for images in tqdm(test_iter):
            with torch.no_grad():
                output = model(images.float().to(CFG["device"]))
                temp_predictions.extend(output.cpu().numpy())   
        all_predictions.append(temp_predictions)
else:
    model.load_state_dict(torch.load(os.path.join(save_path, f'{CFG["model"]}.pth')))
    model.eval()
    temp_predictions = []
    for images in tqdm(test_iter):
        with torch.no_grad():
            output = model(images.float().to(CFG["device"]))
            temp_predictions.extend(output.cpu().numpy())   
    all_predictions.append(temp_predictions)
    
all_predictions = np.array(all_predictions)
all_predictions = all_predictions.sum(axis=0)
all_predictions = all_predictions.argmax(axis=-1)
            
submission['ans'] = all_predictions
# submission.to_csv(f'/opt/ml/code/submissions/test_lsloss.csv', index=False)
submission.to_csv(f'/opt/ml/code/submissions/final_{CFG["model"]}.csv', index=False)

 85%|████████▌ | 336/394 [00:49<00:08,  6.75it/s]